In [2]:
#Import libraries
import numpy as np
import os
import pickle as pickle
from itertools import permutations
from cymetric.pointgen.pointgen import PointGenerator
from cymetric.models.tfhelper import prepare_tf_basis
import tensorflow as tf
tfk = tf.keras

In [3]:
#Set up point generator
n_coords = 5
monomial_powers = 5
monomials = monomial_powers*np.eye(n_coords, dtype=np.int64)
coefficients = np.ones(n_coords)
kmoduli = np.ones(1)
ambient = np.array([n_coords-1])
#print(monomials,coefficients,kmoduli,ambient)
pg = PointGenerator(monomials, coefficients, kmoduli, ambient) ###...better to import this directly somehow?

#Generate points
n_pts = 1000
dirname = 'cy_models/link_data'
kappa = pg.prepare_dataset(n_pts, dirname, val_split=0.)
data = np.load(os.path.join(dirname, 'dataset.npz'))
pg.prepare_basis(dirname, kappa=kappa)


/Users/ed/miniforge3/envs/cymetric/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/ed/miniforge3/envs/cymetric/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
pointgen:INFO:Vol_k: 5.0, Vol_cy: 287.45090973298755.


0

In [4]:
#Import the pre-trained CYMetric model
#Set up the generalte data as cymetric processable objects
from cymetric.models.tfhelper import prepare_tf_basis
dirname = 'cy_models/link_data'
data = np.load(os.path.join(dirname, 'dataset.npz'))
points = data['X_train']
BASIS = np.load(os.path.join(dirname, 'basis.pickle'), allow_pickle=True)
BASIS = prepare_tf_basis(BASIS)

#Reinstate model structure
n_in = 2*5
n_out = 1
nlayer = 3
nHidden = 64
act = 'gelu'
alpha = [1., 1., 1., 1., 1.]
nn_phi = tfk.Sequential()
nn_phi.add(tfk.Input(shape=(n_in)))
for i in range(nlayer):
    nn_phi.add(tfk.layers.Dense(nHidden, activation=act))
nn_phi.add(tfk.layers.Dense(n_out, use_bias=False))

#Import the model
from cymetric.models.tfmodels import PhiFSModel
cymodel_filepath = '/Users/ed/Documents/Projects/G2/Flows/G2Metric/github/cy_models/test_model.keras'
cymetric_model = PhiFSModel(nn_phi, BASIS, alpha=alpha)
cymetric_model.nn_phi = tfk.models.load_model(cymodel_filepath)
#cymetric_model.summary()


In [5]:
#Define necessary functions
def CoordChange_C5R10(point, inverse=False):
    ## TO DO: vectorise for many points
    if not inverse:
        #return np.stack((point.real, point.imag), axis=-1).reshape(-1) #...form of (x1, y1, x2, y2, ...)
        return np.concatenate((np.real(point),np.imag(point))) #...form of (x1, x2, ..., y1, y2, ...)
    else:
        #return point[::2] + 1j * point[1::2] #...form of (x1, y1, x2, y2, ...)
        return point[:len(point)//2] + 1j * point[len(point)//2:] #...form of (x1, x2, ..., y1, y2, ...)
        
        
def hermitian_to_real_symmetric(H):
    """
    Convert a 3x3 Hermitian matrix (complex) to a 6x6 real symmetric matrix
    corresponding to real coordinates (x1, x2, x3, y1, y2, y3).
    
    Parameters:
        H (np.ndarray): A 3x3 Hermitian matrix (complex dtype)

    Returns:
        G (np.ndarray): A 6x6 real symmetric matrix (real dtype)
    """
    if H.shape != (3, 3):
        raise ValueError("Input matrix must be 3x3.")
    if not np.allclose(H, H.conj().T):
        raise ValueError("Input matrix must be Hermitian.")
    
    A = H.real
    B = H.imag
    
    upper = np.hstack((A, -B))
    lower = np.hstack((B,  A))
    G = np.vstack((upper, lower))
    
    # Reordering: from (x1, y1, x2, y2, x3, y3) → (x1, x2, x3, y1, y2, y3)
    perm = [0, 2, 4, 1, 3, 5]
    G_reordered = G[np.ix_(perm, perm)]
    
    return G_reordered


def kahler_form_real(H):
    """
    Given a 3x3 Hermitian matrix H (complex metric),
    return the 6x6 real antisymmetric matrix representing the Kähler form
    in real coordinates: (x1, x2, x3, y1, y2, y3).
    
    Parameters:
        H (np.ndarray): A 3x3 Hermitian matrix

    Returns:
        omega (np.ndarray): 6x6 real antisymmetric Kähler form matrix
    """
    if H.shape != (3, 3):
        raise ValueError("Input must be a 3x3 matrix.")
    if not np.allclose(H, H.conj().T):
        raise ValueError("Matrix must be Hermitian.")

    A = H.real  # Real part of the Hermitian metric
    zero = np.zeros_like(A)
    
    omega = np.block([
        [ zero,  A ],
        [ -A.T, zero ]
    ])
    
    # Reordering: from (x1, y1, x2, y2, x3, y3) → (x1, x2, x3, y1, y2, y3)
    perm = [0, 2, 4, 1, 3, 5]
    omega_reordered = omega[np.ix_(perm, perm)]
    
    return omega_reordered


def wedge_form2_with_form1(omega6, alpha7):
    """
    Compute the wedge product of a 6x6 antisymmetric 2-form with a 1-form of length 7.
    Returns a 7x7x7 antisymmetric 3-form as a NumPy array.
    """
    assert omega6.shape == (6, 6)
    assert np.allclose(omega6, -omega6.T), "omega6 must be antisymmetric"
    assert alpha7.shape == (7,), "alpha7 must be a 7D 1-form"

    omega7 = np.zeros((7, 7))
    omega7[:6, :6] = omega6

    # Create the 3-form tensor
    form3 = np.zeros((7, 7, 7))

    # Compute (omega ∧ alpha)_{ijk} = omega_{ij} * alpha_k + cyclic perms
    for i in range(7):
        for j in range(7):
            for k in range(7):
                val = (1./3.)*(
                    omega7[i, j] * alpha7[k] +
                    omega7[j, k] * alpha7[i] +
                    omega7[k, i] * alpha7[j]
                ) #...antisym part is implicit here as omega7[i, j] = -omega7[j, i], so these not repeated and the 1/3!=1/6 --> 1/3
                form3[i, j, k] = val
    return form3


def holomorphic_volume_form_to_real_tensor(c):
    """
    Convert a complex coefficient c of dz^1 ^ dz^2 ^ dz^3 into a real 6x6x6 tensor 
    representing the real coordinate expression of the holomorphic volume form.
    
    Parameters:
        c (complex): the coefficient of dz^1 ^ dz^2 ^ dz^3
    
    Returns:
        Omega_real (np.ndarray): real part of the 3-form as a 6x6x6 tensor
        Omega_imag (np.ndarray): imaginary part of the 3-form as a 6x6x6 tensor
    """
    # Index mapping
    dx_indices = [0,1,2] #[0, 2, 4]
    dy_indices = [3,4,5] #[1, 3, 5]

    # Prepare output tensors
    Omega_real = np.zeros((6, 6, 6))
    Omega_imag = np.zeros((6, 6, 6))

    # Terms in dz^1 ^ dz^2 ^ dz^3: (dx^i + i dy^i) ^ (dx^j + i dy^j) ^ (dx^k + i dy^k)
    for i, ix in enumerate(dx_indices):
        for j, jx in enumerate(dx_indices):
            if j == i: continue
            for k, kx in enumerate(dx_indices):
                if k == i or k == j: continue

                # Corresponding y indices
                iy, jy, ky = dy_indices[i], dy_indices[j], dy_indices[k]

                # Get all 8 combinations of dx/dy for dz1 ^ dz2 ^ dz3
                for a in [0, 1]:
                    for b in [0, 1]:
                        for d in [0, 1]:
                            coeff = (1j)**(a + b + d) * c / 6  # 6 accounts for antisymmetrization
                            idx = [ix, jx, kx]
                            idx[0] = [ix, iy][a]
                            idx[1] = [jx, jy][b]
                            idx[2] = [kx, ky][d]

                            # antisymmetrize over permutations
                            for perm in permutations(idx):
                                sign = (
                                    1 if list(perm) == sorted(perm) else
                                    (-1) ** len([1 for a, b in zip(perm, sorted(perm)) if a > b])
                                )
                                Omega_real[perm] += (coeff.real * sign)
                                Omega_imag[perm] += (coeff.imag * sign)

    return Omega_real, Omega_imag


In [8]:
#Define the class for the link points
class LinkPoint:
    def __init__(self, cy_point):
        self.cy_point = cy_point
        self.one_idx = np.where(np.isclose(self.cy_point,complex(1,0)))[0][0] #...find the index which is set to 1 to define the patch
        self.dropped_idx = pg._find_max_dQ_coords([self.cy_point])[0]  #...find the index where the CY eqn is subbed, (note pg is external)
        
        self.cy_point_in_S9 = self.cy_point/np.linalg.norm(self.cy_point) 
        self.theta = np.random.uniform(low=0., high=2*np.pi) #...sample a random angle
        self.link_point = self.cy_point_in_S9 * np.exp(1j * self.theta) #...multiple that angle in, the plane origin is the normalised cy point and orientation is implicitly set
        self.dtheta = self.cy_point_in_S9 * 1j * np.exp(1j * self.theta) #...the position vector 90degrees around the S1 is parallel to this d\theta, and cause its on sphere it is normalised already
        #self.dtheta_R1 = ### --> this is just self.theta?
        
        c3_coords = np.delete(self.link_point,[self.one_idx, self.dropped_idx]) #...produce the local complex CY coordinates
        self.link_point_local = np.concatenate((np.real(c3_coords), np.imag(c3_coords), np.array([self.theta]))) #...generate the link local coordinates
        
        self.holomorphic_volume_form = pg.holomorphic_volume_form(self.cy_point) # (note pg is external)
        self.hvf_r, self.hvf_i = holomorphic_volume_form_to_real_tensor(self.holomorphic_volume_form)

        self.hermitian_metric = cymetric_model(CoordChange_C5R10(self.cy_point).reshape(1, 10)).numpy().reshape((3,3)) 
        self.riemannian_metric_R6 = hermitian_to_real_symmetric(self.hermitian_metric)
        self.kahler_form_R6 = kahler_form_real(self.hermitian_metric)
        self.phi_R7 = wedge_form2_with_form1(self.kahler_form_R6, np.concatenate((np.zeros(6),[self.theta])))
        self.phi_R7[:6, :6, :6] += self.hvf_i
        
        pass


In [11]:
#Print an example point
point_idx = 100
test_point = LinkPoint(CoordChange_C5R10(points[point_idx],inverse=True))

print(f'Example point: {np.round(test_point.link_point_local, 3)}\n')
print(f'Respective 3-form:\n{np.round(test_point.phi_R7, 3)}')


Example point: [-0.281  0.337 -0.014 -0.141  0.262  0.418  3.575]

Respective 3-form:
[[[ 0.     0.     0.     0.     0.     0.     0.   ]
  [ 0.     0.     0.008  0.     0.     0.095  0.   ]
  [ 0.    -0.008  0.     0.     0.095  0.     0.021]
  [ 0.     0.     0.     0.     0.     0.     0.   ]
  [ 0.     0.    -0.095  0.     0.    -0.008  0.095]
  [ 0.    -0.095  0.     0.     0.008  0.     0.002]
  [ 0.     0.    -0.021  0.    -0.095 -0.002  0.   ]]

 [[ 0.     0.    -0.008  0.     0.    -0.095  0.   ]
  [ 0.     0.     0.     0.     0.     0.     0.   ]
  [ 0.008  0.     0.     0.095  0.     0.    -0.027]
  [ 0.     0.    -0.095  0.     0.    -0.008  0.   ]
  [ 0.     0.     0.     0.     0.     0.     0.002]
  [ 0.095  0.     0.     0.008  0.     0.     0.126]
  [ 0.     0.     0.027  0.    -0.002 -0.126  0.   ]]

 [[ 0.    -0.008  0.     0.    -0.095  0.    -0.021]
  [-0.008  0.     0.    -0.095  0.     0.     0.027]
  [ 0.     0.     0.     0.     0.     0.     0.   ]
  [ 0.   